In [1]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 5.2 MB/s 
     |████████████████████████████████| 453 kB 53.4 MB/s 


# TF 구현

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7f191b105910>)

In [59]:
train_data = pd.read_table('./ratings_train.txt')
test_data = pd.read_table('./ratings_test.txt')

In [60]:
train_data.drop_duplicates(subset=['document'], inplace=True)

In [61]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [7]:
train_data

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [62]:
train_data['document'] = train_data['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
train_data['document'].replace('', np.nan, inplace=True)
train_data = train_data.dropna(how = 'any')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [63]:
test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['document'] = test_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

전처리 후 테스트용 샘플의 개수 : 48852


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
okt = Okt()
okt.morphs('와 이런 것도 영화라고 차라리 뮤직비디오를 만드는 게 나을 뻔', stem = True)

['오다', '이렇다', '것', '도', '영화', '라고', '차라리', '뮤직비디오', '를', '만들다', '게', '나다', '뻔']

In [9]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [20]:
X_train = []
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    status = ""
    for i in stopwords_removed_sentence:
      status += i + " "
    X_train.append(status)

100%|██████████| 146182/146182 [08:55<00:00, 273.01it/s]


In [21]:
X_test = []
for sentence in tqdm(test_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    status = ""
    for i in stopwords_removed_sentence:
      status += i + " "
    X_test.append(status)

100%|██████████| 48852/48852 [03:20<00:00, 243.64it/s]


In [22]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [23]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [24]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 47481
등장 빈도가 2번 이하인 희귀 단어의 수: 27240
단어 집합에서 희귀 단어의 비율: 57.3703165476717
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 2.032442597015398


In [25]:
# 전체 단어 개수 중 빈도수 2이하인 단어는 제거.
# 0번 패딩 토큰을 고려하여 + 1
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 20242


In [26]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

In [27]:
X_train = np.delete(X_train, drop_train, axis=0)
print(len(X_train))

146180


In [28]:
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
max_len = 50

In [ ]:
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [19]:
X_train

[[48, 461, 15, 255, 658],
 [965, 471, 41, 609, 1, 220, 1391, 23, 960, 692, 20],
 [395, 2641, 3721, 6576, 2, 226, 9],
 [5818, 106, 8612, 222, 57, 4, 25, 3860],
 [1094,
  29,
  9905,
  23,
  837,
  1,
  2765,
  21,
  1148,
  252,
  13969,
  19487,
  1082,
  258,
  252],
 [778,
  5663,
  1026,
  1364,
  138,
  433,
  146,
  1813,
  62,
  1680,
  12879,
  481,
  233,
  1,
  92,
  121,
  1067,
  46,
  268],
 [225, 317, 3, 343, 494],
 [121,
  1067,
  46,
  333,
  24,
  11920,
  17019,
  326,
  124,
  1606,
  368,
  328,
  236,
  10,
  730,
  17,
  581,
  572,
  529,
  478,
  3221,
  8613,
  14,
  1403,
  1403,
  37,
  287,
  4,
  23,
  34,
  39,
  14,
  706,
  1103,
  71],
 [95, 4, 57, 6, 368, 99, 1],
 [1498, 25, 204, 557, 82, 14, 393, 1483, 369, 659, 8, 5307, 6],
 [4884, 165, 5, 15, 165, 5, 510],
 [763, 400, 1201, 209, 1155, 1174, 2488, 904, 12880, 628, 2617, 797, 5537, 5],
 [2182, 144, 887, 44, 9430, 47, 1232, 281, 952, 780, 4045, 23, 1033, 34],
 [2562, 1269, 7, 470, 1193, 124, 732, 1523, 

In [31]:
text = ["와 영화 개노잼이네 진짜 망할듯",
        "시간 가는 줄 모르고 재밌게 봤습니다.",
        "ㄹㅇ 개노잼"]

In [47]:
text_processing = []
for i in text:
    tokenized_sentence = okt.morphs(i, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    status = ""
    for i in stopwords_removed_sentence:
      status += i + " "
    text_processing.append(status)

In [48]:
text_processing = tokenizer.texts_to_sequences(text_processing)

In [55]:
X_test[:50]

[[750, 88],
 [60, 164, 26, 446, 20, 18, 327, 888, 47, 796, 19],
 [65, 20, 91, 345, 111, 108, 62, 155, 280],
 [14, 19, 124, 2287, 104, 62, 36, 17, 24, 193, 14250, 813],
 [210, 2997, 11, 28, 210, 1],
 [670, 69],
 [1036, 385, 330, 21, 4337, 7508, 4, 1461, 4201, 2146, 5528, 3707],
 [526,
  1200,
  11,
  2824,
  212,
  57,
  4,
  3151,
  677,
  201,
  44,
  103,
  2241,
  212,
  48,
  397,
  15864,
  15,
  1,
  86,
  369,
  48,
  2230,
  5],
 [2652, 1406, 495, 3152, 555, 3093, 4024, 231, 41, 74, 20],
 [138, 347, 14, 7609, 403, 56],
 [200, 1135, 1149, 525, 704, 11, 621, 11],
 [1072,
  149,
  29,
  149,
  248,
  6160,
  3,
  2701,
  428,
  9446,
  149,
  248,
  3,
  63,
  15956,
  1488,
  149,
  10649,
  15,
  3749,
  1,
  5],
 [238, 245, 190, 5, 1, 9799, 374, 1017, 20],
 [33,
  115,
  23,
  1377,
  1104,
  599,
  1,
  1361,
  981,
  103,
  1,
  159,
  766,
  3,
  448,
  43,
  1,
  241,
  53,
  27,
  1,
  6111,
  378,
  119,
  27,
  1,
  3967,
  15,
  4435,
  317],
 [6675, 2260, 1240, 3005, 2

In [56]:
test_size = 50
text_score = [0 for i in range(test_size)]

for i in tqdm(range(len(X_test[:test_size]))):
  for j in X_train:
    union = set(X_test[i]).union(set(j))
    intersection = set(X_test[i]).intersection(set(j))
    text_score[i] = max(text_score[i], len(intersection) / len(union))

100%|██████████| 50/50 [00:28<00:00,  1.77it/s]


In [67]:
test_data.head(30)

,id,document,label
0,6270596,굳 ㅋ,1
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0
5,7898805,음악이 주가 된 최고의 음악영화,1
6,6315043,진정한 쓰레기,0
7,6097171,마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가고개를 젖게한다,0
8,8932678,갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한남...,0
9,6242223,이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 모든 사람이 그렇지는 않네,1
10,7462111,괜찮네요오랜만포켓몬스터잼밌어요,1


In [69]:
test_data_head = test_data.head(50)
test_data_head['score'] = text_score

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [70]:
test_data_head

,id,document,label,score
0,6270596,굳 ㅋ,1,1.000000
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0,0.384615
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0,0.400000
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0,0.294118
5,7898805,음악이 주가 된 최고의 음악영화,1,0.428571
6,6315043,진정한 쓰레기,0,0.666667
7,6097171,마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가고개를 젖게한다,0,0.157895
8,8932678,갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한남...,0,0.200000
9,6242223,이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 모든 사람이 그렇지는 않네,1,0.235294
10,7462111,괜찮네요오랜만포켓몬스터잼밌어요,1,0.333333


In [57]:
text_score

[1.0,
 0.38461538461538464,
 0.4,
 0.29411764705882354,
 0.42857142857142855,
 0.6666666666666666,
 0.15789473684210525,
 0.2,
 0.23529411764705882,
 0.3333333333333333,
 0.25,
 0.21052631578947367,
 0.2727272727272727,
 0.1935483870967742,
 0.3333333333333333,
 1.0,
 0.375,
 1.0,
 0.3,
 0.15,
 0.23809523809523808,
 0.6666666666666666,
 0.6666666666666666,
 0.15,
 0.15555555555555556,
 0.3333333333333333,
 0.17647058823529413,
 0.3,
 0.6,
 1.0,
 0.5,
 0.21428571428571427,
 0.5,
 0.42857142857142855,
 0.4,
 1.0,
 0.3333333333333333,
 0.2,
 0.3333333333333333,
 0.25,
 0.5,
 0.75,
 0.5,
 0.3333333333333333,
 1.0,
 0.25,
 0.12244897959183673,
 0.2857142857142857,
 0.25,
 0.36363636363636365]

# Doc2Vec

In [71]:
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument
from konlpy.tag import Mecab

In [ ]:
model = doc2vec.Doc2Vec(vector_size=vocab_size, alpha=0.025, min_alpha=0.025, workers=8, window=8)

# Vocabulary 빌드
model.build_vocab(tagged_corpus_list)
print(f"Tag Size: {len(model.docvecs.doctags.keys())}", end=' / ')

# Doc2Vec 학습
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=50)

# 모델 저장
model.save('./dart.doc2vec')

# TF, TF-IDF -> 램 부족으로 유기

In [41]:
tfvector = CountVectorizer()
tfvector.fit(X_train)
# 코퍼스로부터 각 단어의 빈도수를 기록

# 각 단어와 맵핑된 인덱스 출력
# print(vector.vocabulary_)

CountVectorizer()

In [ ]:
tfvector.fit_transform(X_train)

In [15]:
tfidf = TfidfVectorizer()
tfidf.fit(X_train)
x_tfidf = tfidf.fit_transform(X_train)

In [18]:
len(X_train)

195034

In [ ]:
x_tfidf.A

In [ ]:
tfidf.transform(X_train).toarray()

# 가짜 리뷰(비슷한 리뷰 자카드 유사도 활용) ㄹㅇ 프로토타입



In [ ]:
text = ["와 영화 개노잼이네 진짜 망할듯",
        "시간 가는 줄 모르고 재밌게 봤습니다.",
        "ㄹㅇ 개노잼"]

In [ ]:
text_processing = []
for i in text:
    tokenized_sentence = okt.morphs(i, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    status = ""
    for i in stopwords_removed_sentence:
      status += i + " "
    text_processing.append(status)

In [ ]:
text_processing = tokenizer.texts_to_sequences(text_processing)

In [ ]:
text_score = [0 for i in range(len(text))]

for i in range(len(text_processing)):
  for j in X_train:
    union = set(text_processing[i]).union(set(j))
    intersection = set(text_processing[i]).intersection(set(j))
    text_score[i] = max(text_score[i], len(intersection) / len(union))

In [ ]:
text_score

[0.5, 0.8333333333333334, 0.6666666666666666]

# 리뷰 요약

In [ ]:
pip install transformers

In [ ]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')